In [1]:
import re
import requests
from datetime import datetime, timedelta
from dateutil import parser
import pytz

In [2]:
lp_table = '\n'.join(filter(lambda l:l[0]!='#',requests.get("https://www.ietf.org/timezones/data/leap-seconds.list").text.split('\n')[:-1]))
print(lp_table)

2272060800	10	# 1 Jan 1972
2287785600	11	# 1 Jul 1972
2303683200	12	# 1 Jan 1973
2335219200	13	# 1 Jan 1974
2366755200	14	# 1 Jan 1975
2398291200	15	# 1 Jan 1976
2429913600	16	# 1 Jan 1977
2461449600	17	# 1 Jan 1978
2492985600	18	# 1 Jan 1979
2524521600	19	# 1 Jan 1980
2571782400	20	# 1 Jul 1981
2603318400	21	# 1 Jul 1982
2634854400	22	# 1 Jul 1983
2698012800	23	# 1 Jul 1985
2776982400	24	# 1 Jan 1988
2840140800	25	# 1 Jan 1990
2871676800	26	# 1 Jan 1991
2918937600	27	# 1 Jul 1992
2950473600	28	# 1 Jul 1993
2982009600	29	# 1 Jul 1994
3029443200	30	# 1 Jan 1996
3076704000	31	# 1 Jul 1997
3124137600	32	# 1 Jan 1999
3345062400	33	# 1 Jan 2006
3439756800	34	# 1 Jan 2009
3550089600	35	# 1 Jul 2012
3644697600	36	# 1 Jul 2015
3692217600	37	# 1 Jan 2017


In [3]:
rx=re.compile("(\d+).*(\d\d).*(\d).*(Jan|Jul).*(\d{4})\n?")
rx.findall(lp_table)

[('2272060800', '10', '1', 'Jan', '1972'),
 ('2287785600', '11', '1', 'Jul', '1972'),
 ('2303683200', '12', '1', 'Jan', '1973'),
 ('2335219200', '13', '1', 'Jan', '1974'),
 ('2366755200', '14', '1', 'Jan', '1975'),
 ('2398291200', '15', '1', 'Jan', '1976'),
 ('2429913600', '16', '1', 'Jan', '1977'),
 ('2461449600', '17', '1', 'Jan', '1978'),
 ('2492985600', '18', '1', 'Jan', '1979'),
 ('2524521600', '19', '1', 'Jan', '1980'),
 ('2571782400', '20', '1', 'Jul', '1981'),
 ('2603318400', '21', '1', 'Jul', '1982'),
 ('2634854400', '22', '1', 'Jul', '1983'),
 ('2698012800', '23', '1', 'Jul', '1985'),
 ('2776982400', '24', '1', 'Jan', '1988'),
 ('2840140800', '25', '1', 'Jan', '1990'),
 ('2871676800', '26', '1', 'Jan', '1991'),
 ('2918937600', '27', '1', 'Jul', '1992'),
 ('2950473600', '28', '1', 'Jul', '1993'),
 ('2982009600', '29', '1', 'Jul', '1994'),
 ('3029443200', '30', '1', 'Jan', '1996'),
 ('3076704000', '31', '1', 'Jul', '1997'),
 ('3124137600', '32', '1', 'Jan', '1999'),
 ('33450624

In [4]:
last_entry=parser.parse(' ' .join(rx.findall(lp_table)[-1][2:]))
last_entry

datetime.datetime(2017, 1, 1, 0, 0)

In [5]:
def direct_table(lp_table):
    _1900=datetime(1900,1,1, tzinfo=pytz.UTC).timestamp()
    res = """constexpr std::array leap_seconds_tt2000 = 
    {"""
    for sec in rx.findall(lp_table):
        offset = int(sec[0])
        corr = int(sec[1])
        ns = int((_1900+offset)*1e9)
        res += f"// {sec[2:]}\n"
        res += f"    std::pair{{ {ns}  , {int(corr*1e9)}}},\n"
    res += "};\n"
    return res

def reverse_table(lp_table):
    tt2000_offset = 946727967816000000
    _1900=datetime(1900,1,1, tzinfo=pytz.UTC).timestamp()
    res = """constexpr std::array leap_seconds_tt2000_reverse = 
    {"""
    for sec in rx.findall(lp_table):
        offset = int(sec[0])
        corr = int(int(sec[1])*1e9)
        ns = int((_1900+offset)*1e9)-tt2000_offset+corr
        res += f"// {sec[2:]}\n"
        res += f"    std::pair{{ {ns}  , {corr}}},\n"
    res += "};\n"
    return res


In [6]:
with open('../include/cdfpp/chrono/cdf-leap-seconds.h', 'w') as leap_file:
    leap_file.write("""
#pragma once
/*------------------------------------------------------------------------------
-- This file is a part of the CDFpp library
-- Copyright (C) 2022, Plasma Physics Laboratory - CNRS
--
-- This program is free software; you can redistribute it and/or modify
-- it under the terms of the GNU General Public License as published by
-- the Free Software Foundation; either version 2 of the License, or
-- (at your option) any later version.
--
-- This program is distributed in the hope that it will be useful,
-- but WITHOUT ANY WARRANTY; without even the implied warranty of
-- MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
-- GNU General Public License for more details.
--
-- You should have received a copy of the GNU General Public License
-- along with this program; if not, write to the Free Software
-- Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA 02111-1307 USA
-------------------------------------------------------------------------------*/
/*-- Author : Alexis Jeandet
-- Mail : alexis.jeandet@member.fsf.org
----------------------------------------------------------------------------*/
#include <chrono>
#include <utility>
#include <array>
namespace cdf::chrono::leap_seconds
{    
using namespace std::chrono;
""")
    leap_file.write(f"""
    static inline constexpr uint32_t last_updated = {last_entry.year}{last_entry.month:02d}{last_entry.day:02d};

    """)
    leap_file.write(direct_table(lp_table))
    leap_file.write(reverse_table(lp_table))
    leap_file.write("""
} //namespace cdf
    """)